# Configuration combining 

## Block extractor 

In this section we would like to performe all the block extract functions from both title and description so we can use to get the configuration set. First we import the dataset.

In [2]:
## read the json file
import json
a = open('C:\\\\Users\\\\mingh\\\\Desktop\\\\text\\\\Master\\\\Thesis\\\\Python Code\\\\Algorithm 4\\\\amazon.json')
amazon_data = json.load(a)

b = open('C:\\\\Users\\\\mingh\\\\Desktop\\\\text\\\\Master\\\\Thesis\\\\Python Code\\\\Algorithm 4\\\\bestbuy.json')
bestbuy_data = json.load(b)

c = open('C:\\\\Users\\\\mingh\\\\Desktop\\\\text\\\\Master\\\\Thesis\\\\Python Code\\\\Algorithm 4\\\\newegg.json')
newegg_data = json.load(c)

d = open('C:\\\\Users\\\\mingh\\\\Desktop\\\\text\\\\Master\\\\Thesis\\\\Python Code\\\\Algorithm 4\\\\thenerds.json')
thenerds_data = json.load(d)

data = {'amazon.com':amazon_data,'bestbuy.com':bestbuy_data,'newegg.com':newegg_data,'thenerds.net':thenerds_data}

## read the json file
f = open('C:\\\\Users\\\\mingh\\\\Desktop\\\\text\\\\Master\\\\Thesis\\\\Python Code\\\\Algorithm 3\\\\matched_mape.json')
aligned_sheet = json.load(f)

In [3]:
# define the transformer function
def block_transformer(block,k):
    if k==1:
        return block
    else:    
        from itertools import combinations
        block_list = []
        for i in combinations(block,k):
            block_list.append(list(i))
        return block_list

In [4]:
# define the modelword check function
def modelwords_check(inputString):
    digit =  any(char.isdigit() for char in inputString)
    if inputString.isnumeric():
        non_digit = False
    else: 
        non_digit = True
    
    return digit & non_digit

In [69]:
test = [(1,2),(3,4),(5,6)]
test = block_transformer(test,2)
for i in test:
    print(str(i))

[(1, 2), (3, 4)]
[(1, 2), (5, 6)]
[(3, 4), (5, 6)]


### Title source

#### Word extractor

In [8]:
def word_title(data,k):
    Title_word_blocks = {}

    # build the blocks
    for shop in data.keys():
        for product in data[shop].values():
            title = product['Title'].split()
            title = block_transformer(title,k)
            for word in title:
                if ','.join(word) in Title_word_blocks.keys():
                    Title_word_blocks[','.join(word)].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:
                    Title_word_blocks[','.join(word)] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]



    # prepare the parameter that we used for evaluation
    block_count_word = len(Title_word_blocks.keys())
    avg_product_count_word = 0
    max_product_count_word = 0
    for block in Title_word_blocks.values():
        avg_product_count_word += len(block)

        if len(block)>max_product_count_word:
            max_product_count_word = len(block)
        else: pass

    avg_product_count_word = avg_product_count_word/block_count_word
    
    return(Title_word_blocks,block_count_word,avg_product_count_word)

#### Modelword extractor

In [11]:
def modelword_title(data,k):
    import re
    Title_modelword_blocks = {}

    # build the blocks
    for shop in data.keys():
        for product in data[shop].values():
            title_raw = product['Title'].split()
            title = []
            for word in title_raw:
                if modelwords_check(word):
                    title.append(word)
                else: pass
            title = block_transformer(title,k)
            for word in title:
                if ','.join(word) in Title_modelword_blocks.keys():
                    Title_modelword_blocks[','.join(word)].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:
                    Title_modelword_blocks[','.join(word)] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]




    # prepare the parameter that we used for evaluation
    block_count_modelword = len(Title_modelword_blocks.keys())
    avg_product_count_modelword = 0
    max_product_count_modelword = 0
    for block in Title_modelword_blocks.values():
        avg_product_count_modelword += len(block)

        if len(block)>max_product_count_modelword:
            max_product_count_modelword = len(block)
        else: pass

    avg_product_count_modelword = avg_product_count_modelword/block_count_modelword
    return(Title_modelword_blocks,block_count_modelword,avg_product_count_modelword)

#### Q-gram extractor 

In [6]:
def qgram_title(data,n = 3,k=1):
    from nltk import ngrams
    # define the required value
    Title_qgram_blocks = {}

    # build the blocks
    for shop in data.keys():
        for product in data[shop].values():
            title = ngrams(product['Title'].split(), n)
            title = block_transformer(title,k)
            for word in title:
                if word[0] in Title_qgram_blocks.keys():
                    Title_qgram_blocks[','.join(word[0])].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:
                    Title_qgram_blocks[','.join(word[0])] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]



    # prepare the parameter that we used for evaluation
    block_count_qgram = len(Title_qgram_blocks.keys())
    avg_product_count_qgram = 0
    max_product_count_qgram = 0
    for block in Title_qgram_blocks.values():
        avg_product_count_qgram += len(block)

        if len(block)>max_product_count_qgram:
            max_product_count_qgram = len(block)
        else: pass

    avg_product_count_qgram = avg_product_count_qgram/block_count_qgram
    
    return(Title_qgram_blocks,block_count_qgram)


#### Skip-gram extractor

In [7]:
def skipgram_title(data,window_size = 2):
    from itertools import combinations
    import string

    # define the required value
    Title_skip_blocks = {}
    # build the blocks
    for shop in data.keys():
        for product in data[shop].values():
            # here we build the skip-gram combinations
            pairs_for_all_sentences = []
            title_clean = product['Title'].translate(str.maketrans('', '', string.punctuation))
            title = [title_clean.split()]
            for sentence in title:
                this_sentence_pairs = []

                for index, target_word in enumerate(sentence): 
                    window_words = sentence[max(index - window_size, 0) : min(index + window_size + 1, len(sentence))]

                    for window_word in window_words:
                        if window_word != target_word:
                            this_sentence_pairs.append((window_word, target_word))

                pairs_for_all_sentences.append(this_sentence_pairs)
            title = pairs_for_all_sentences[0]


            for word in title:
                if word in Title_skip_blocks.keys():
                    Title_skip_blocks[word].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:
                    Title_skip_blocks[word] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]



    # prepare the parameter that we used for evaluation
    block_count_skip = len(Title_skip_blocks.keys())
    avg_product_count_skip = 0
    max_product_count_skip = 0
    for block in Title_skip_blocks.values():
        avg_product_count_skip += len(block)

        if len(block)>max_product_count_skip:
            max_product_count_skip = len(block)
        else: pass

    avg_product_count_skip = avg_product_count_skip/block_count_skip
    
    return(Title_skip_blocks,block_count_skip)

#### Suffix array extractor

In [8]:
def suffix_title(data):
    # define the required value
    Title_suffix_blocks = {}

    # build the blocks
    for shop in data.keys():
        for product in data[shop].values():
            title = product['Title'].split()
            title = sorted([title[i:], i] for i in range(len(title)))
            for suffix in title:
                suffix_str = str(suffix)
                if suffix_str in Title_suffix_blocks.keys():
                    Title_suffix_blocks[suffix_str].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:
                    Title_suffix_blocks[suffix_str] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]



    # prepare the parameter that we used for evaluation
    block_count_suffix = len(Title_suffix_blocks.keys())
    avg_product_count_suffix = 0
    max_product_count_suffix = 0
    for block in Title_suffix_blocks.values():
        avg_product_count_suffix += len(block)

        if len(block)>max_product_count_suffix:
            max_product_count_suffix = len(block)
        else: pass

    avg_product_count_suffix = avg_product_count_suffix/block_count_suffix
    
    return(Title_suffix_blocks,block_count_suffix)

### Description source

In [9]:
# A function to find all the matched keys of a specific key
def matched_key(shop1,key1):
    key_set = [key1]
    for shop in aligned_sheet[shop1][key1].values():
        if shop['score']>0.28:
            key_set.append(shop['key'])
            
    key_set.sort()        
    return(key_set)

#### Word extractor

In [71]:
def word_KVP(data,k=1):
    # define the required value
    KVP_word_blocks = {}

    # build the blocks
    for shop in data.keys():
        for product in data[shop].values():
            key_value = []
            for value in product['featuresMap'].keys():
                value_split = product['featuresMap'][value].split()  
                block = block_transformer(value_split,k)
                for word in block:
                    if word in key_value:
                        pass
                    else:
                        key_value.append(word)
                        
            for word in key_value:
                if str(word) in KVP_word_blocks.keys():

                    KVP_word_blocks[str(word)].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:

                    KVP_word_blocks[str(word)] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]


    # prepare the parameter that we used for evaluation
    block_count_word = len(KVP_word_blocks.keys())
    avg_product_count_word = 0
    max_product_count_word = 0
    for block in KVP_word_blocks.values():
        avg_product_count_word += len(block)

        if len(block)>max_product_count_word:
            max_product_count_word = len(block)
        else: pass

    avg_product_count_word = avg_product_count_word/block_count_word
    return(KVP_word_blocks,block_count_word)

#### Modelword extractor

In [70]:
def modelword_KVP(data,k=1):
    # define the required value
    KVP_modelword_blocks = {}

    # build the blocks
    for shop in data.keys():
        rest_shop = list(data.keys()).remove(shop)
        for product in data[shop].values():
            modelword_list = []
            key_value = []
            for value in product['featuresMap'].keys():
                value_block = []
                value_split = product['featuresMap'][value].split()  
                for word in value_split:
                    if modelwords_check(word):
                        modelword_list.append(word)
                
                modelword_list = block_transformer(modelword_list,k)
                for block in modelword_list:
                    if block in key_value:
                        pass
                    else:
                        key_value.append(block)
                        
            for word in key_value:
                if str(word) in KVP_modelword_blocks.keys():

                    KVP_modelword_blocks[str(word)].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:

                    KVP_modelword_blocks[str(word)] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]



    # prepare the parameter that we used for evaluation
    block_count_modelword = len(KVP_modelword_blocks.keys())
    avg_product_count_modelword = 0
    max_product_count_modelword = 0
    for block in KVP_modelword_blocks.values():
        avg_product_count_modelword += len(block)

        if len(block)>max_product_count_modelword:
            max_product_count_modelword = len(block)
        else: pass

    avg_product_count_modelword = avg_product_count_modelword/block_count_modelword
    return(KVP_modelword_blocks,block_count_modelword)

#### Q-gram extractor

In [12]:
def qgram_KVP(data):
    from nltk import ngrams

    # define the required value
    KVP_qgram_blocks = {}
    n=3

    # build the blocks
    for shop in data.keys():
        for product in data[shop].values():
            key_value = []
            for value in product['featuresMap'].keys():
                value_split = ngrams(product['featuresMap'][value].split(),n)
                for ngram in value_split:
                    key_value.append(ngram)

            for word in key_value:
                if ','.join(word) in KVP_qgram_blocks.keys():
                    KVP_qgram_blocks[','.join(word)].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:
                    KVP_qgram_blocks[','.join(word)] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]


    # prepare the parameter that we used for evaluation
    block_count_qgram = len(KVP_qgram_blocks.keys())
    avg_product_count_qgram = 0
    max_product_count_qgram = 0
    for block in KVP_qgram_blocks.values():
        avg_product_count_qgram += len(block)

        if len(block)>max_product_count_qgram:
            max_product_count_qgram = len(block)
        else: pass

    avg_product_count_qgram = avg_product_count_qgram/block_count_qgram
    
    return(KVP_qgram_blocks,block_count_qgram)



#### Skip-gram extractor

In [13]:
def skipgram_KVP(data):
    from itertools import combinations
    import string

    # define the required value
    KVP_skip_blocks = {}
    window_size = 2 # here we define the length of the window we would focus when we extract information from the strings

    # build the blocks
    for shop in data.keys():
        for product in data[shop].values():
            # here we build the skip-gram combinations
            pairs_for_all_sentences = []
            key_value = []
            for value in product['featuresMap'].keys():
                value_clean = product['featuresMap'][value].translate(str.maketrans('', '', string.punctuation))
                value_split = [value_clean.split()]
                for word in value_split:
                    key_value.append(word)

            for sentence in key_value:
                this_sentence_pairs = []

            for index, target_word in enumerate(sentence): 
                window_words = sentence[max(index - window_size, 0) : min(index + window_size + 1, len(sentence))]
                for window_word in window_words:
                    if window_word != target_word:
                        this_sentence_pairs.append((window_word, target_word))

            pairs_for_all_sentences.append(this_sentence_pairs)

            for i in pairs_for_all_sentences:
                for combination in i:
                    key_value.append([value,str(combination)])


            for word in key_value:
                if ','.join(word) in KVP_skip_blocks.keys():

                    KVP_skip_blocks[','.join(word)].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:

                    KVP_skip_blocks[','.join(word)] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]



    # prepare the parameter that we used for evaluation
    block_count_skip = len(KVP_skip_blocks.keys())
    avg_product_count_skip = 0
    max_product_count_skip = 0
    for block in KVP_skip_blocks.values():
        avg_product_count_skip += len(block)

        if len(block)>max_product_count_skip:
            max_product_count_skip = len(block)
        else: pass

    avg_product_count_skip = avg_product_count_skip/block_count_skip
    
    return(KVP_skip_blocks,block_count_skip)

#### Suffix array extractor

In [14]:
def suffix_KVP(data):
    # define the required value
    KVP_suffix_blocks = {}

    # build the blocks
    for shop in data.keys():
        for product in data[shop].values():
            key_value = []
            for value in product['featuresMap'].values():
                value_split = value.split()
                value_split = sorted([value_split[i:], i] for i in range(len(value_split)))
                for word in value_split:
                    key_value.append(word)

            for suffix in key_value:
                suffix_str = str(suffix)
                if suffix_str in KVP_suffix_blocks.keys():
                    KVP_suffix_blocks[suffix_str].append({'title':product['Title'],'shop':shop,'ID':product['model_ID']})
                else:
                    KVP_suffix_blocks[suffix_str] = [{'title':product['Title'],'shop':shop,'ID':product['model_ID']}]



    # prepare the parameter that we used for evaluation
    block_count_suffix = len(KVP_suffix_blocks.keys())
    avg_product_count_suffix = 0
    max_product_count_suffix = 0
    for block in KVP_suffix_blocks.values():
        avg_product_count_suffix += len(block)

        if len(block)>max_product_count_suffix:
            max_product_count_suffix = len(block)
        else: pass

    avg_product_count_suffix = avg_product_count_suffix/block_count_suffix
    return(KVP_suffix_blocks,block_count_suffix)

## Data merger

### Combine with the original data

In this section we merge the result of the blocking schemas with the original data, so as the result each product in the original data should contains one more variable called 'matched product'.

In [357]:
# Initical a dataset based on the block schema

def data_initical(block_schema):
    import copy
    output_data = copy.deepcopy(data)
    
    for shop in output_data.keys():
        for product_this in output_data[shop].values():
            product_this['matched product'] = []
            for block in block_schema.keys():
                block_check = False

                for product_block in block_schema[block]:
                    if product_this['Title'] == product_block['title'] and product_this['model_ID'] == product_block['ID'] and shop == product_block['shop'] :
                        block_check = True


                if block_check == True:
                    for i in block_schema[block]:
                        if i['shop'] == shop:
                            pass
                        else:   # in case we count same product several times as paired product
                            if i in product_this['matched product']:
                                pass
                            else:
                                product_this['matched product'].append(i)
                                
                
    return(output_data)


### Block aggregation 

#### Union 

In [377]:
def Union(dataset_1,dataset_2):
    import copy
    Output = copy.deepcopy(dataset_1)
    
    for shop in Output.keys():
        for product_this in Output[shop].values():
            matched_pair_1 = list(product_this['matched product'])
            matched_pair_2 = list(dataset_2[shop][product_this['model_ID']]['matched product'])
            matched_pair = matched_pair_1 + matched_pair_2
            uniq = []
            [uniq.append(x) for x in matched_pair if x not in uniq]

            product_this['matched product'] = uniq
            
    return(Output)

#### Conjunction

In [378]:
# we first set a function to find the common element in two list
def common_elements(list1, list2):
    return [element for element in list1 if element in list2]

In [379]:
def Conjunction(dataset_1,dataset_2):
    import copy
    Output = copy.deepcopy(dataset_1)
    
    for shop in Output.keys():
        for product_this in Output[shop].values():
            
            matched_pair_1 = list(product_this['matched product'])
            matched_pair_2 = list(dataset_2[shop][product_this['model_ID']]['matched product'])
                
            
            product_this['matched product'] = common_elements(matched_pair_1,matched_pair_2)
            
    return(Output)
    

##  Configuration building

Now we propose the possible configuration that we would like to further discuss

In [393]:
configuration_low = [['t_mw',3,'d_skip','conjunction'],['t_mw',3,'d_suffix','conjunction'],['t_mw',4,'d_qgram','union'],['t_wo',3,'d_wo','conjunction'],['t_mw',3,'d_mw','conjunction']]
configuration_mid = [['t_mw',2,'d_skip','conjunction'],['t_mw',3,'d_qgram','union'],['t_wo',4,'d_skip','conjunction'],['t_wo',3,'d_mw','conjunction'],['t_mw',2,'d_wo','conjunction'],['t_qgram','d_mw','conjunction']]
configuration_high = [['t_mw',2,'d_skip','union'],['t_mw',2,'d_qgram','union'],['t_wo',2,'d_suffix','union'],['t_mw',3,'d_wo','union'],['t_skip','d_wo','union']]



In [394]:
for i in configuration_3:
    if i[0] == 't_mw':
        result1,count1 = modelword_title(data,i[1])
    elif i[0] == 't_wo':
        result1,count1 = word_title(data,i[1])
    elif i[0] == 't_skip':
        result1,count1 = skipgram_title(data)
        
    if i[2] == 'd_mw':
        result2,count2 = modelword_KVP(data)
    elif i[2] == 'd_wo':
        result2,count2 = word_KVP(data)
        

    import json
    name_union = i[0]+str(i[1])+'_'+i[2] +'_union.json'   
    name_conjunction = i[0]+str(i[1])+'_'+i[2] +'_conjunction.json'
    with open(name_union, 'w') as u:
        json.dump(union_set, u)    
    with open(name_conjunction, 'w') as c:
        json.dump(conjunction_set, c)
        


In [ ]:


for i in configuration_2:
    if i[0] == 't_qgram':
        result1,count1 = skipgram_title(data)
    elif i[0] == 't_suffix':
        result1,count1 = suffix_title(data)
    elif i[0] == 't_skip':
        result1,count1 = qgram_title(data)
        
    if i[1] == 'd_mw':
        result2,count2 = modelword_KVP(data)
    elif i[1] == 'd_wo':
        result2,count2 = word_KVP(data)
        
    data_title = data_initical(result1)
    data_KVP = data_initical(result2)


    conjunction_set_2 = Conjunction(data_title,data_KVP)
        
    name_conjunction_2 = i[0]+'_'+i[1] +'_conjunction.json'
    with open(name_conjunction_2, 'w') as c:
        json.dump(conjunction_set_2, c)

# Test area

In [12]:
import datetime
tstart = datetime.datetime.now()


result1,count1,avg_product_count_modelword = modelword_title(data,4)

tend = datetime.datetime.now()
duration = tend - tstart
duration


datetime.timedelta(seconds=1, microseconds=659714)

In [13]:
avg_product_count_modelword

1.3188303681521931

In [383]:
import datetime
tstart = datetime.datetime.now()


result2,count2 = modelword_title(data,2)

tend = datetime.datetime.now()
duration = tend - tstart
duration
count2

5049

In [236]:
duration

datetime.timedelta(microseconds=250028)

### merged data

### Aggregation 

In [370]:
data_title = data_initical(result1)
data_KVP = data_initical(result2)

t_wo2_d_skip_union = Union(data_title,data_KVP)
t_wo2_d_skip_conjunction = Conjunction(data_title,data_KVP)

import json

with open('t_wo2_d_skip_union.json', 'w') as u:
    json.dump(t_wo2_d_skip_union, u)
    
with open('t_wo2_d_skip_conjunction.json', 'w') as c:
    json.dump(t_wo2_d_skip_conjunction, c)

In [220]:
t_wo2_d_qgram_union['amazon.com']['UN46ES6580']

{'Title': 'Samsung UN46ES6580 46-Inch 1080p 120Hz 3D Slim LED HDTV (Black)',
 'featuresMap': {'Date first available at Amazon.com': 'February 20, 2012',
  'Product Dimensions:': '52 x 6.8 x 28.7 inches ; 33.3 pounds',
  'Display Technology': 'LED-lit',
  'Display Size': '46 inches',
  'Shipping:': 'Currently, item can be shipped only within the U.S.',
  'Brand Name': 'Samsung',
  'Image Aspect Ratio': '16:09',
  'ASIN:': 'B007B9PMCO',
  'Shipping Weight': '41 pounds ('},
 'brand': 'Samsung',
 'model_ID': 'UN46ES6580',
 'matched product': [{'title': 'Samsung 32" Class 3112" Diag. LED 1080p 60Hz HDTV UN32F5000AFXZA - Best Buy',
   'shop': 'bestbuy.com',
   'ID': 'UN32F5000AFXZA'},
  {'title': 'Samsung - 46" Class / LED / 1080p / 120Hz / 3D / HDTV',
   'shop': 'bestbuy.com',
   'ID': 'UN46D6900W'},
  {'title': 'Panasonic - 50" Class / Plasma / 720p / 600Hz / HDTV',
   'shop': 'bestbuy.com',
   'ID': 'TC-P50X3'},
  {'title': 'Samsung 32" Class 3112" Diag. LED 720p 60Hz HDTV UN32EH4003FXZA 

In [384]:
data_title = data_initical(result1)
data_KVP = data_initical(result2)

t_wo2_d_skip_union = Union(data_title,data_KVP)
t_wo2_d_skip_conjunction = Conjunction(data_title,data_KVP)

In [385]:
len(t_wo2_d_skip_union['amazon.com']['UN46ES6580']['matched product'])

500

In [386]:
len(t_wo2_d_skip_conjunction['amazon.com']['UN46ES6580']['matched product'])

68